In [1]:
import pandas as pd
import glob
import warnings
warnings.filterwarnings('ignore')
from functools import reduce
import re
import datetime

In [2]:
DemandTracker=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\DemandTracker.xlsx')
HCL=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\HCL.xlsx')
INFY=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\Infy.xlsx')
OnboardingTracker=pd.read_excel(r'C:\NotBackedUp\SOE\pyscript\data\OnboardingTracker.xlsx')

Candidate throughput rate is important (as per the contract)

In [3]:
hclcandidates = HCL[['Title','Tribe','Sourcing Partner','Role','Candidate Type','Application Date','Hacker Rank Test Date','ANZ Interview Date','Application Status'
]]
Infycandidates = INFY[['Title','Tribe','Sourcing Partner','Role','Candidate Type','Application Date','Hacker Rank Test Date','ANZ Interview Date','Application Status'
]]

In [4]:
DRO = DemandTracker[['Demand ID','Demand Status','Tribe','Role','Start Date','Partner','Resource Name','Created']]

In [5]:
hclc= hclcandidates.merge(DRO,how='left',left_on=['Title'],right_on=['Demand ID'])
infyc=Infycandidates.merge(DRO,how='left',left_on=['Title'],right_on=['Demand ID'])

In [6]:
HCLNo_Candidate = hclc.groupby(['Demand ID'])['Demand ID'].count().reset_index(name='ToTal_Candidates')
INFYNo_Candidate = infyc.groupby(['Demand ID'])['Demand ID'].count().reset_index(name='ToTal_Candidates')

In [7]:
failHackRankHCL = hclc[hclc['Application Status']=='Failed Hacker Rank']
failHackRankHCL = failHackRankHCL.groupby(['Demand ID'])['Demand ID'].count().reset_index(name='FailedHackerRank')

failHackRankInfy = infyc[infyc['Application Status']=='Failed Hacker Rank']
failHackRankInfy = failHackRankInfy.groupby(['Demand ID'])['Demand ID'].count().reset_index(name='FailedHackerRank')

In [8]:
failInterviewHCL =hclc[hclc['Application Status']=='Failed Interview']
failInterviewHCL = failInterviewHCL.groupby(['Demand ID'])['Demand ID'].count().reset_index(name='FailedInterview')

failInterviewInfy =infyc[infyc['Application Status']=='Failed Interview']
failInterviewInfy = failInterviewInfy.groupby(['Demand ID'])['Demand ID'].count().reset_index(name='FailedInterview')

In [9]:
HiredHCL =hclc[hclc['Application Status']=='Hired']
HiredHCL = HiredHCL.groupby(['Demand ID'])['Demand ID'].count().reset_index(name='Hired')

HiredInfy =infyc[infyc['Application Status']=='Hired']
HiredInfy = HiredInfy.groupby(['Demand ID'])['Demand ID'].count().reset_index(name='Hired')

In [10]:
PendingHackerRankHCL =hclc[hclc['Application Status']=='Pending Hacker Rank']
PendingHackerRankHCL = PendingHackerRankHCL.groupby(['Demand ID'])['Demand ID'].count().reset_index(name='PendingHackerRank')
PendingHackerRankInfy =infyc[infyc['Application Status']=='Pending Hacker Rank']
PendingHackerRankInfy = PendingHackerRankInfy.groupby(['Demand ID'])['Demand ID'].count().reset_index(name='PendingHackerRank')

In [11]:
hclstats = [HCLNo_Candidate,failHackRankHCL,failInterviewHCL,HiredHCL,PendingHackerRankHCL]
infystats = [INFYNo_Candidate,failHackRankInfy,failInterviewHCL,HiredInfy,PendingHackerRankInfy]

In [12]:
HCLthroughput = reduce(lambda left,right: pd.merge(left,right,on=['Demand ID'],how='outer'),hclstats)
Infythroughput = reduce(lambda left,right: pd.merge(left,right,on=['Demand ID'],how='outer'),infystats)

In [13]:
HCLthroughput[['ToTal_Candidates','FailedHackerRank','FailedInterview','Hired','PendingHackerRank']]=HCLthroughput[['ToTal_Candidates','FailedHackerRank','FailedInterview','Hired','PendingHackerRank']].fillna(0)
HCLthroughput[['ToTal_Candidates','FailedHackerRank','FailedInterview','Hired','PendingHackerRank']]=HCLthroughput[['ToTal_Candidates','FailedHackerRank','FailedInterview','Hired','PendingHackerRank']].astype(int)

Infythroughput[['ToTal_Candidates','FailedHackerRank','FailedInterview','Hired','PendingHackerRank']]=Infythroughput[['ToTal_Candidates','FailedHackerRank','FailedInterview','Hired','PendingHackerRank']].fillna(0)
Infythroughput[['ToTal_Candidates','FailedHackerRank','FailedInterview','Hired','PendingHackerRank']]=Infythroughput[['ToTal_Candidates','FailedHackerRank','FailedInterview','Hired','PendingHackerRank']].astype(int)

In [14]:
#HCLthroughput.to_csv(r'C:\NotBackedUp\SOE\pyscript\HCLCandidates.csv',sep=',',index=None)
#Infythroughput.to_csv(r'C:\NotBackedUp\SOE\pyscript\InfyCandidates.csv',sep=',',index=None)

-% of internal referrals per role type is important


In [15]:
Internalhcl = HCL[HCL['Candidate Type']=='Internal Candidate']
InternalInfy = INFY[INFY['Candidate Type']=='Internal Candidate']

In [16]:
roles = DemandTracker
roles.loc[roles['Role'].str.contains('Engineer - Tester'),'Role'] = 'Tester'
roles.loc[roles['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
roles.loc[roles['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
roles.loc[roles['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
roles = roles.groupby(['Role'])['Demand ID'].count().reset_index(name='RolesCount')

In [17]:
Externalhcl = HCL[HCL['Candidate Type']=='External Candidate']
ExternalInfy = INFY[INFY['Candidate Type']=='External Candidate']

In [18]:
Internalhcl.loc[Internalhcl['Role'].str.contains('Engineer - Tester'),'Role'] = 'Tester'
Internalhcl.loc[Internalhcl['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
Internalhcl.loc[Internalhcl['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
Internalhcl.loc[Internalhcl['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'

InternalInfy.loc[InternalInfy['Role'].str.contains('Engineer - Tester'),'Role'] = 'Tester'
InternalInfy.loc[InternalInfy['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
InternalInfy.loc[InternalInfy['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
InternalInfy.loc[InternalInfy['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'

In [19]:
Externalhcl.loc[Externalhcl['Role'].str.contains('Engineer - Tester'),'Role'] = 'Tester'
Externalhcl.loc[Externalhcl['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
Externalhcl.loc[Externalhcl['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
Externalhcl.loc[Externalhcl['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'

ExternalInfy.loc[ExternalInfy['Role'].str.contains('Engineer - Tester'),'Role'] = 'Tester'
ExternalInfy.loc[ExternalInfy['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
ExternalInfy.loc[ExternalInfy['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
ExternalInfy.loc[ExternalInfy['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'

In [20]:
HCLInt = Internalhcl.groupby(['Role'])['Title'].count().reset_index(name='HCLInternalReferrals')
INFYInt = InternalInfy.groupby(['Role'])['Title'].count().reset_index(name='InfyInternalReferrals')
HCLExt = Externalhcl.groupby(['Role'])['Title'].count().reset_index(name='HCLExternalReferrals')
INFYExt = ExternalInfy.groupby(['Role'])['Title'].count().reset_index(name='InfyExternalReferrals')

In [21]:
HCLCandicate = HCL
HCLCandicate.loc[HCLCandicate['Role'].str.contains('Engineer - Tester'),'Role'] = 'Tester'
HCLCandicate.loc[HCLCandicate['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
HCLCandicate.loc[HCLCandicate['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
HCLCandicate.loc[HCLCandicate['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
HCLCandicate=HCLCandicate.groupby(['Role'])['ID'].count().reset_index(name='TotalHCL')

In [22]:
INFYCandicate = INFY
INFYCandicate.loc[INFYCandicate['Role'].str.contains('Engineer - Tester'),'Role'] = 'Tester'
INFYCandicate.loc[INFYCandicate['Role'].str.contains('Engineer'),'Role'] = 'Engineer'
INFYCandicate.loc[INFYCandicate['Role'].str.contains('Expert'),'Role'] = 'Operations Expert'
INFYCandicate.loc[INFYCandicate['Role'].str.contains('Partner'),'Role'] = 'Delivery Partner'
INFYCandicate=INFYCandicate.groupby(['Role'])['ID'].count().reset_index(name='TotalINFY')

In [23]:
refere = [roles,HCLInt,INFYInt,HCLExt,INFYExt,HCLCandicate,INFYCandicate]

In [24]:
InternalRefferals = reduce(lambda left,right: pd.merge(left,right,on=['Role'],how='outer'),refere)

In [25]:
InternalRefferals

,Role,RolesCount,HCLInternalReferrals,InfyInternalReferrals,HCLExternalReferrals,InfyExternalReferrals,TotalHCL,TotalINFY
0,Agile Coach,1,1.0,3.0,14.0,12.0,15.0,15.0
1,Analyst,50,8.0,12.0,23.0,29.0,31.0,41.0
2,Developer,23,NaN,NaN,1.0,NaN,1.0,NaN
3,Engineer,131,43.0,52.0,132.0,236.0,175.0,288.0
4,Environment Manager,2,NaN,NaN,NaN,NaN,NaN,NaN
5,Operations Expert,4,1.0,1.0,NaN,2.0,1.0,3.0
6,Product Owner,1,NaN,NaN,NaN,NaN,NaN,NaN
7,Release Lead,36,6.0,2.0,24.0,18.0,30.0,20.0
8,SDET - Automation Tester,6,NaN,NaN,NaN,NaN,NaN,NaN
9,Scrum Master,5,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
InternalRefferals[['HCLInternalReferrals','InfyInternalReferrals','HCLExternalReferrals','InfyExternalReferrals','TotalHCL','TotalINFY']]=InternalRefferals[['HCLInternalReferrals','InfyInternalReferrals','HCLExternalReferrals','InfyExternalReferrals','TotalHCL','TotalINFY']].fillna(0)
InternalRefferals[['HCLInternalReferrals','InfyInternalReferrals','HCLExternalReferrals','InfyExternalReferrals','TotalHCL','TotalINFY']]=InternalRefferals[['HCLInternalReferrals','InfyInternalReferrals','HCLExternalReferrals','InfyExternalReferrals','TotalHCL','TotalINFY']].astype(int)

In [27]:
InternalRefferals

,Role,RolesCount,HCLInternalReferrals,InfyInternalReferrals,HCLExternalReferrals,InfyExternalReferrals,TotalHCL,TotalINFY
0,Agile Coach,1,1,3,14,12,15,15
1,Analyst,50,8,12,23,29,31,41
2,Developer,23,0,0,1,0,1,0
3,Engineer,131,43,52,132,236,175,288
4,Environment Manager,2,0,0,0,0,0,0
5,Operations Expert,4,1,1,0,2,1,3
6,Product Owner,1,0,0,0,0,0,0
7,Release Lead,36,6,2,24,18,30,20
8,SDET - Automation Tester,6,0,0,0,0,0,0
9,Scrum Master,5,0,0,0,0,0,0


In [28]:
InternalRefferals.to_csv(r'C:\NotBackedUp\SOE\pyscript\Referals.csv',sep=',',index=None)